## Step 1 - Obtain Initial Hartree-Fock Solution

In [ ]:
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.drivers import PySCFDriver

# Define the PySCF driver to simulate an H_2 molecule 
# with the hydrogen atoms distanced 0.735 ANGSTROM units apart
driver = PySCFDriver(
    atom="H 0 0 0; H 0 0 0.735",
    basis="sto3g",
    charge=0,
    spin=0,
    unit=DistanceUnit.ANGSTROM,
)

In [ ]:
# Run the SCF driver to compute the electronic-hamiltonian for the system above.
# The hamiltonian is used in the eigenvalue minimization to find the ground state of the system.
problem = driver.run()
print(problem)

Show the terms in the 1 and 2-body integrals

$h_{pq} = \int \phi^*_p(r) \left( -\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{R_I- r} \right)   \phi_q(r)dr$
<br>
$h_{pqrs} = \int \frac{\phi^*_p(r_1)  \phi^*_q(r_2) \phi_r(r_2)  \phi_s(r_1)}{|r_1-r_2|}dr_1dr_2$


Here, we show the values of the integrals for electrons with a-spin. You can think of a-spin and b-spin as being "spin up" and "spin down".

The Hartree-Fock solution for the system's hamiltonian is defined as the following: <br>
$\hat{H}_{elec}=\sum_{pq} h_{pq} \hat{a}^{\dagger}_p \hat{a}_q + \frac{1}{2} \sum_{pqrs} h_{pqrs}  \hat{a}^{\dagger}_p \hat{a}^{\dagger}_q \hat{a}_r  \hat{a}_s$

The 1-body integrals describe the electron's energy without considering interactions with other electrons. This would include things like its kinetic energy.<br>
$h_{pq} = \int \phi^*_p(r) \left( -\frac{1}{2} \nabla^2 - \sum_{I} \frac{Z_I}{R_I- r} \right)   \phi_q(r)dr$

The 2-body integrals describe energy resulting from interactions between bodies, like electron-electron interactions causing repulsion.<br>
$h_{pqrs} = \int \frac{\phi^*_p(r_1)  \phi^*_q(r_2) \phi_r(r_2)  \phi_s(r_1)}{|r_1-r_2|}dr_1dr_2$

In [14]:
hamiltonian = problem.hamiltonian

coefficients = hamiltonian.electronic_integrals
# Since H_2 is closed-shell, the alpha and beta coefficients should be the same. Here we inspect the alpha terms.
# The alpha-beta interactions in the 2-body integrals are accounted for in the hamiltonian, but we don't inspect them here since they're derived
# from the alpha and beta terms which are equal to eachother.
print(coefficients.alpha) 

Polynomial Tensor
 "+-":
array([[-8.39667704e-01,  8.26334066e-17],
       [ 7.51088373e-17, -6.72280171e-01]])
 "++--":
array([ 5.29139733e-01, -5.30876774e-17,  2.44180554e-01,  5.38388757e-01,
        6.50905963e-17,  5.58067043e-01])


The following sections shows the second-quantized operator.

In [18]:
second_q_op = hamiltonian.second_q_op()
print(second_q_op)

Fermionic Operator
number spin orbitals=4, number terms=36
  -0.8396677036664356 * ( +_0 -_0 )
+ -0.672280170773492 * ( +_1 -_1 )
+ -0.8396677036664356 * ( +_2 -_2 )
+ -0.672280170773492 * ( +_3 -_3 )
+ 0.2645698662904512 * ( +_0 +_0 -_0 -_0 )
+ 0.2691943787365952 * ( +_0 +_1 -_1 -_0 )
+ 0.2645698662904512 * ( +_0 +_2 -_2 -_0 )
+ 0.2691943787365952 * ( +_0 +_3 -_3 -_0 )
+ 0.12209027710142221 * ( +_0 +_0 -_1 -_1 )
+ 0.12209027710142221 * ( +_0 +_1 -_0 -_1 )
+ 0.12209027710142221 * ( +_0 +_2 -_3 -_1 )
+ 0.12209027710142221 * ( +_0 +_3 -_2 -_1 )
+ 0.12209027710142221 * ( +_1 +_0 -_1 -_0 )
+ 0.12209027710142221 * ( +_1 +_1 -_0 -_0 )
+ 0.12209027710142221 * ( +_1 +_2 -_3 -_0 )
+ 0.12209027710142221 * ( +_1 +_3 -_2 -_0 )
+ 0.2691943787365952 * ( +_1 +_0 -_0 -_1 )
+ 0.27903352127802156 * ( +_1 +_1 -_1 -_1 )
+ 0.2691943787365952 * ( +_1 +_2 -_2 -_1 )
+ 0.27903352127802156 * ( +_1 +_3 -_3 -_1 )
+ 0.2645698662904512 * ( +_2 +_0 -_0 -_2 )
+ 0.2691943787365952 * ( +_2 +_1 -_1 -_2 )
+ 0.26456986629

## Step 2 - Compute the ground state energy level of $H_2$
Now that we have Hartree-Fock Hamiltonian of the $H_2$ molecule, we can use qiskit's ground state eigensolver to compute / approximate the ground state energy level.

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver
from qiskit_nature.second_q.algorithms import GroundStateEigensolver
from qiskit_nature.second_q.mappers import JordanWignerMapper

solver = GroundStateEigensolver(
    JordanWignerMapper(),
    NumPyMinimumEigensolver(),
)

In [ ]:
# Solve for the minimum eigenvalue of the hamiltonian
result = solver.solve(problem)
print(result)

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -1.272801376405
  - computed part:      -1.272801376405
~ Nuclear repulsion energy (Hartree): 0.305001274305
> Total ground state energy (Hartree): -0.967800102099
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [0.0  0.0  3.27867483]
 
  0: 
  * Electronic dipole moment (a.u.): [0.0  0.0  3.27867482612]
    - computed part:      [0.0  0.0  3.27867482612]
  > Dipole moment (a.u.): [0.0  0.0  0.00000000388]  Total: 0.00000000388
                 (debye): [0.0  0.0  0.000000009861]  Total: 0.000000009861
 
